In [ ]:
!pip install transformers
!pip install mosestokenizer
!pip install sentencepiece

In [ ]:
!git clone

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sentencepiece as spm
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
pd.set_option('display.max_rows', 500)

In [ ]:

def download(model_name):
  tokenizer = MarianTokenizer.from_pretrained(model_name)
  model = MarianMTModel.from_pretrained(model_name)
  return tokenizer, model
# download model for English -> Romance
tmp_lang_tokenizer, tmp_lang_model = download('Helsinki-NLP/opus-mt-en-ROMANCE')
# download model for Romance -> English
src_lang_tokenizer, src_lang_model = download('Helsinki-NLP/opus-mt-ROMANCE-en')

In [ ]:
def translate(texts, model, tokenizer, language):
  """Translate texts into a target language"""
  # Format the text as expected by the model
  formatter_fn = lambda txt: f"{txt}" if language == "en" else f">>{language}<< {txt}"
  original_texts = [formatter_fn(txt) for txt in texts]
  translated = model.generate(**tokenizer(original_texts, return_tensors="pt", padding=True), temperature=1.0)
  res = []
  for t in translated:
      res.append(tokenizer.decode(t, skip_special_tokens=True))
  return res

def back_translate(texts, language_src, language_dst):
  """Implements back translation"""
  # Translate from source to target language
  translated = translate(texts, tmp_lang_model, tmp_lang_tokenizer, language_dst)

  # Translate from target language back to source language
  back_translated = translate(translated, src_lang_model, src_lang_tokenizer, language_src)

  return back_translated

In [ ]:
src_texts = ['I might be late tonight', 'What a movie, so bad', 'That was very kind']
back_texts = back_translate(src_texts, "en", "fr")

print(back_texts)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Je pourrais être en retard ce soir.
Quel film, si mauvais
C'était très gentil.
I might be late tonight.
What a movie, so bad
That was very kind of you.
['I might be late tonight.', 'What a movie, so bad', 'That was very kind of you.']


In [ ]:
import pandas as pd

label2idx = {'admiration': 0, 'amusement': 1, 'anger': 2,
      'annoyance': 3, 'approval': 4, 'caring': 5,
      'confusion': 6, 'curiosity': 7, 'desire': 8,
      'disappointment': 9, 'disapproval': 10, 'disgust': 11,
      'embarrassment': 12, 'excitement': 13, 'fear': 14,
      'gratitude': 15, 'grief': 16, 'joy': 17,
      'love': 18, 'nervousness': 19, 'optimism': 20,
      'pride': 21, 'realization': 22, 'relief': 23,
      'remorse': 24, 'sadness': 25, 'surprise': 26}
idx2label = {}
for key, val in label2idx.items():
    idx2label[val] = key
train_data['true_label'] = train_data['label'].replace(idx2label)

train_data_sub = train_data[train_data['true_label'].isin(['annoyance', 'approval', 'caring', 'confusion', 'desire', 'disappointment', 'disapproval', 'excitement', 'nervousness', 'grief', 'pride', 'realization', 'relief'])]
df = train_data_sub.reset_index(drop = True)

NameError: ignored

In [ ]:
train_data_sub = train_data[train_data['true_label'].isin(['annoyance', 'approval', 'caring', 'confusion', 'desire', 'disappointment', 'disapproval', 'excitement', 'nervousness', 'grief', 'pride', 'realization', 'relief'])]

In [ ]:
def process_row(row):
    original_text = row['text']
    back_translated_text = back_translate([original_text], 'en', 'fr')[0]
    if original_text != back_translated_text:
        # Create a new row with the same data but with the back-translated text
        new_row = row.copy()
        new_row['text'] = back_translated_text
        return new_row

# Iterate over the DataFrame and collect new rows
new_rows = []
for index, row in train_data_sub.iterrows():
    new_row = process_row(row)
    if new_row is not None:
        new_rows.append(new_row)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
On faisait beaucoup de merde dans le passé, ce n'est pas un bon argument.
We used to do a lot of shit in the past, that's not a good argument.
C'est une sattive ou une indica?
Is it a sattive or an indica?
il s'agit d'une estimation fiable. cela pourrait se produire dès les années 2020, si elle était malchanceuse
this is a reliable estimate. this could happen as early as the 2020s, if it was bad luck
J'aime ça.
I like that.
Je me souviens d'avoir dormi sur la fenêtre arrière "ledge" dans la voiture de mes parents! Je n'y ai pas pensé depuis des années.
I remember sleeping on the back window "ledge" in my parents' car! I haven't thought about it in years.
Oui, ça prend du temps, mais utilisez toute cette énergie négative et concentrez-vous sur vous-même.
Yes, it takes time, but use all that negative energy and focus on yourself.
Pourquoi j'aurais pu poster quelque chose sur les mecs et faire semblant d'être la femme qui parle merde de m

NameError: ignored

In [ ]:
# Create a DataFrame from the new rows and concatenate it with the original DataFrame
if new_rows:
    new_rows_df = pd.DataFrame(new_rows)
    new_train_data = pd.concat([train_data, new_rows_df], ignore_index=True)

In [ ]:
train_data_drop = train_data_sub.drop_duplicates(subset='text', keep="first")
train_data_sub.to_csv("train_data.csv")